In [1]:
from core.PipelineProject import PrepareDataAndTrainingModels
from core.SQLeto import SQLeto
from decouple import config

In [2]:
RAW_DATA_PATH = config('RAW_DATA_PATH')
DATABASE_POSTGRES = config('DATABASE_POSTGRES')
USER_POSTGRES = config('USER_POSTGRES')
PWD_POSTGRES = config('PWD_POSTGRES')

In [3]:
# Create object to access database
sqleto = SQLeto(
    database=DATABASE_POSTGRES, 
    user=USER_POSTGRES,
    password=PWD_POSTGRES)
    
# Getting data in a dataframe
data = sqleto.execute_DQL(
    '''
        SELECT * FROM creditcard
    '''
)

In [4]:
# Models to test
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
# Scaler
from sklearn.preprocessing import RobustScaler
# Balancer
from imblearn.over_sampling import SMOTE


In [5]:
prep = PrepareDataAndTrainingModels(
    dataframe=data,
    target="Class",
    scaler=RobustScaler(),
    models=[
        DecisionTreeClassifier(),
        MLPClassifier(),
        LogisticRegression(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        XGBClassifier()

    ],
    balancer = SMOTE(),
    score_metric = 'recall_macro'
)

In [6]:
prep.avaliate_models_cv()

Its Binary Classification!


##################################################


Using tecnique: SMOTE()
Model training: DecisionTreeClassifier()


Metric: recall_macro - 0.9981985658157904
##################################################
Model training: MLPClassifier()


Metric: recall_macro - 0.9995678546621409
##################################################
Model training: LogisticRegression()




C:\Users\Nophaie\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Nophaie\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

Metric: recall_macro - 0.9772798081086034
##################################################
Model training: RandomForestClassifier()


Metric: recall_macro - 0.9998844266556958
##################################################
Model training: GradientBoostingClassifier()


